* go to the folder python4gate
* save your notebooks (.ipynb) to different filenames
* git pull

# Image (.hdr, .mhd, etc...) manipulation for GATE

## Contents:
* Reading images with NumPy
* Reading images with SimpleITK
* Merging images
* Converting Dicom image serie to MetaImages (.mhd/.raw)

# Reading images

## Reading any binary image
PET mouse data from https://neuroimage.usc.edu/neuro/Digimouse_Download in Analyze (.hdr / .img) format.

In [ ]:
import numpy as np

file = 'data/mouse_PET.img'
data_type = np.float32
dimension = [128, 128, 129]

# read the binary file
image = np.fromfile(file, dtype=data_type)
image = image.reshape(dimension,order='F')

print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[63,:,:], cmap=plt.cm.jet)

In [ ]:
from ipywidgets import interact

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[myslice,:,:], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[0]-1))
    return;

im_viewer(image, plt.cm.jet)

In [ ]:
# save the binary image
image.tofile('output/mouse_PET_modified.img')

## Reading Meta images (.mhd / .raw)
Easy image reading with the library SimpleITK

In [ ]:
import SimpleITK as sitk

# 1) Read the image
itkimage = sitk.ReadImage('data/patient_CT.mhd')
image = sitk.GetArrayFromImage(itkimage)

print ('itkimage.GetSize() :' , itkimage.GetSize())
print ('itkimage.GetSpacing() :' , itkimage.GetSpacing())

print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[140,:,:], cmap=plt.cm.bone)

In [ ]:
# save the image
output = sitk.GetImageFromArray(image)
sitk.WriteImage(output,'output/patient_CT.mhd')

## Exercise
* modify the image data so that the slices of the CT (220) correspond to the last dimension
* crop the image in the coronal plane to 25:110
* save the modified image

In [ ]:
# complete here ...

## Reading Dicom images (.dcm)
Easy image reading with the library SimpleITK

In [ ]:
import SimpleITK as sitk

# 1) Read the image
itkimage = sitk.ReadImage('data/patient_SPECT.dcm')
image = sitk.GetArrayFromImage(itkimage)

print ('itkimage.GetSpacing() :' , itkimage.GetSpacing())
print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[140,:,:], cmap=plt.cm.bone)

## Reading Dicom images (.dcm)
Easy image reading with the library PyDicom

In [ ]:
import pydicom as dcm

# open dicom file (works for all .dcm file)
File = dcm.read_file('data/patient_SPECT.dcm')
# Acces to image pixel data 
image = File.pixel_array

print(type(image))
print('Pixel spacing :', File.PixelSpacing)
print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[140,:,:], cmap=plt.cm.bone)

# Merging images

## Merge GATE output DoseActor .mhd of splitted simulations

In [ ]:
import SimpleITK as sitk
from glob import glob

# directory of your simulation splitted outputs
output_dir = 'output/'

# loop over the .mhd outputs
first_image = True
for file in glob(output_dir+'*Dose.mhd'):
#    if '_merged' not in file:
        print ('Reading file:',file)
        # 1) Read the image
        itkimage = sitk.ReadImage(file)
        # 2) Sum the image values
        if first_image:
            image_merged = sitk.GetArrayFromImage(itkimage)
            first_image = False
        else: 
            image_merged += sitk.GetArrayFromImage(itkimage)
        #print ('Sum of all voxels:',image_merged.max())

# 3) Write the sum of all images
itkimage_merged = sitk.GetImageFromArray(image_merged)
itkimage_merged.CopyInformation(itkimage)
sitk.WriteImage(itkimage_merged, output_dir+'output-Dose_merged.mhd')

print ('Merged file created:', output_dir+'output-Dose_merged.mhd')
print ('Dimension : ' , itkimage_merged.GetSize())
print ('Pixels Size :' , itkimage_merged.GetSpacing())

# Converting DICOM to MetaImage (.mhd/.raw)

In [ ]:
import SimpleITK as sitk

input_dir = 'convert/dicom/'
output_dir = 'convert/mhd/'

series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(input_dir)

if not series_IDs:
    print('No series in directory ' + input_dir)

for series in series_IDs:
    filename = series.split('.')[-1]
    sitk.WriteImage(sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(input_dir, series)),
                    output_dir+filename+'.mhd')
    print('Dicom serie converted to:', output_dir+filename+'.mhd')

## Exercise
* place a copy of the data/patient_CT_dcm/*.dcm images in the folder convert/dicom/  
* convert the Dicom images serie to a single .mhd image using the cell above
* read the converted image and display an axial slice of it

In [ ]:
# complete here ...